# Retrieval API DEMO

## Code Initialisation
Dependencies and environment initialisation.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `env` file as template.

In [1]:
import os
import json
import requests as r
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [2]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_MODEL = os.getenv('GCP_MODEL')

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [3]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown(f"**Authentication Successful**: Bearer token OK for user {USERNAME.split('@')[0].split('-')[0]}"))
else:
    display(Markdown(f"**Authentication Failed**: Cannot obtain the Bearer token for the user {USERNAME.split('@')[0].split('-')[0]}"))

**Authentication Successful**: Bearer token OK for user 9ZZZ159100

In [4]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

## Factiva Retrieval API Query

### Prompt
Add the prompt to be sent to the Retrieval API

In [13]:
# "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
# "中国2025年计划进行哪些月球探测任务？其主要目标是什么？"   # What are China's planned missions to the Moon in 2025, and what are their primary objectives?
# "今後12か月間の日本の半導体業界の見通しはどのようなものですか?"  # What are the perspectives for the semiconductors industry in Japan for the next 12 months?
# "Summarise the latest earnings report from Microsoft Corp"
# "What are the perspectives of bitcoin for the next 12 months?"
frapi_prompt = "Summarise tariff decisions by the US administration during the first week of April 2025 and their impact on the world economy."

Assemble the Retrieval API payload

In [14]:
frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 5,
      "query": {
        "search_filters": [
          {
            "scope": "Language",
            "value": "en"       # en or ja or de or it or es
          }
        ],

        "value": frapi_prompt
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}


## Send Query and Receive Chunks from the Retrieval API

In [15]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print Chunks

In [16]:
chunks_list = chunks_resp.json()['data']

# u.print_full_chunks(chunks_list)
u.print_partial_chunks(chunks_list)

### Trump announces pause in 'reciprocal' tariffs for all countries but China 

**Washington Post.com** - 2025-04-09 - WPCOM00020250409el490018j - Lang: en

President Donald Trump on Wednesday raised tariffs on imports from China to 125 percent while simultaneously pausing many new tariffs on goods from ot...

---

### Stocks sink despite talk of trade deals

...

---

### China increases levies on U.S. as Trump's 104% tariffs take effect; stocks sink 

**Washington Post.com** - 2025-04-09 - WPCOM00020250409el4900001 - Lang: en

Beijing hit back Wednesday against President Donald Trump's tariffs by imposing an 84 percent levy on all U.S. goods, as the world's second-largest ec...

---

## Retrieval API Conclusion

Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Test LLM

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example only a few articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is Google Gemini 2.0 Flash. Response generations took a few seconds for a context between 3K to 4K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Gemini Structured Prompt

In [17]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format:
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [18]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "Summarise tariff decisions by the US administration during the first week of April 2025 and their impact on the world economy.",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.WPCOM00020250409el490018j",
            "source_name": "Washington Post.com",
            "headline": "Trump announces pause in 'reciprocal' tariffs for all countries but China",
            "publication_date": "2025-04-09",
            "content": "President Donald Trump on Wednesday raised tariffs on imports from China to 125 percent while simultaneously pausing many new tariffs on goods from ot..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.WP00000020250409el4900013",
            "source_name": "The Washington Post",
            "headline": "Stocks sink despite talk of trade deals",
            "publication_date": "2025-04-09",
       

### Gemini Request

In [19]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION, GCP_MODEL)
display(Markdown(f"**Prompt:** {frapi_prompt} \n --------------------"))
display(Markdown(response))

**Prompt:** Summarise tariff decisions by the US administration during the first week of April 2025 and their impact on the world economy. 
 --------------------

During the first week of April 2025, the US administration, under President Trump, made several tariff decisions that significantly impacted the world economy.

On April 2nd, President Trump announced massive tariffs on almost every country, effectively the largest U.S. tax increase since 1968 [4]. These tariffs raised the average effective U.S. tariff rate from 2.5% in 2024 to around 22.5% [4]. JPMorgan raised the probability of a global recession to 60% from 40% and expected U.S. GDP to contract 0.3% in the fourth quarter of 2025 [4].

On April 9th, Trump raised tariffs on imports from China to 125 percent while simultaneously pausing many new tariffs on goods from other nations [1]. This move sent the stock market surging after a tumultuous week [1]. However, this pause could compromise the White House's plans to raise $6 trillion in new revenue to fund tax cuts [1].

China retaliated by imposing an 84 percent levy on all U.S. goods [5]. The State Council stated that this was a response to Trump's "Liberation Day" tariffs, which raised U.S. duties on all imports from China to 104 percent [5]. China also expressed a willingness to resolve differences through dialogue and consultation [5].

Other countries also responded with tariffs. Canada's tariffs of 25 percent on U.S. autos took effect, matching U.S. tariffs [1]. The European Union issued tariffs of 25 percent on some U.S. products [1].

The global stock markets reacted negatively to these developments. European markets sold off, with London's FTSE 100, France's CAC index, and Germany's DAX each down roughly 3 percent [1]. In Asia, Japan's Nikkei 225 lost 4 percent, while South Korea's KOSPI ended the day roughly 2 percent lower [1]. Hong Kong's Hang Seng Index initially fell sharply but recovered to a modest gain [1]. The S&P 500 index closed down 1.5 percent, bringing its total loss since mid-February to almost 20 percent, indicating a bear market [2].

Pharmaceutical companies were also affected, with European companies like AstraZeneca and Roche experiencing sell-offs after Trump hinted at impending tariffs on imported pharmaceuticals [1, 5].

Cited Articles:

- [1] [Trump announces pause in 'reciprocal' tariffs for all countries but China - Washington Post.com - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.WPCOM00020250409el490018j)
- [2] [Stocks sink despite talk of trade deals - The Washington Post - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.WP00000020250409el4900013)
- [4] [The Outlook: Tariffs Erase Economy's Solid Outlook -- WSJ - Dow Jones Institutional News - 2025-04-07](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250407el47000dg)
- [5] [China increases levies on U.S. as Trump's 104% tariffs take effect; stocks sink - Washington Post.com - 2025-04-09](https://dj.factiva.com/article?id=drn:archive.newsarticle.WPCOM00020250409el4900001)
